# Question Answering – Évaluation du modèle

Ce notebook évalue les performances du modèle fine-tuné
sur le dataset SQuAD (Exact Match, F1-score et temps d’inférence).


## Objectifs

- Charger le modèle fine-tuné
- Évaluer les performances sur le jeu de validation
- Calculer les métriques Exact Match et F1
- Mesurer le temps d’inférence


In [ ]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import evaluate
import numpy as np
import time
import torch
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import matplotlib.pyplot as plt


## Chargement des données préprocessées


In [ ]:
tokenized_datasets = load_from_disk("outputs/tokenized_squad")


## Chargement du modèle fine-tuné


In [ ]:
model_path = "outputs/checkpoints/roberta/final"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

model.eval()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


## Métriques SQuAD


In [ ]:
metric = evaluate.load("squad")


## Fonction d’inférence


In [ ]:
def predict_with_score(example):
    inputs = {
        "input_ids": torch.tensor(example["input_ids"]).unsqueeze(0).to(device),
        "attention_mask": torch.tensor(example["attention_mask"]).unsqueeze(0).to(device)
    }

    with torch.no_grad():
        outputs = model(**inputs)

    start_logits = outputs.start_logits.squeeze()
    end_logits = outputs.end_logits.squeeze()

    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)

    score = start_logits[start_idx] + end_logits[end_idx]

    return start_idx.item(), end_idx.item(), score.item()


## Évaluation sur le jeu de validation


In [ ]:
n_samples = 500
validation_set = tokenized_datasets["validation"].select(range(n_samples))


In [ ]:
y_true = []
y_scores = []

for example in validation_set:
    start_pred, end_pred, score = predict_with_score(example)

    prediction_text = tokenizer.decode(
        example["input_ids"][start_pred:end_pred + 1],
        skip_special_tokens=True
    )

    gold_text = example["answers"]["text"][0]

    # Exact Match → label binaire
    y_true.append(int(prediction_text.strip() == gold_text.strip()))
    y_scores.append(score)

    metric.add(
        prediction={
            "id": example["id"],
            "prediction_text": prediction_text
        },
        reference={
            "id": example["id"],
            "answers": {
                "text": [gold_text],
                "answer_start": [example["answers"]["answer_start"][0]]
            }
        }
    )


## Résultats


In [ ]:
results = metric.compute()
results


In [ ]:
plt.figure()
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – Question Answering")
plt.legend()
plt.show()


## Visualisation de la courbe ROC


In [ ]:
fpr, tpr, roc_thresholds = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)

roc_auc


## Courbe ROC


In [ ]:
precision, recall, pr_thresholds = precision_recall_curve(y_true, y_scores)

precision[:5], recall[:5]


## Precision et Recall


## Mesure du temps d’inférence


In [ ]:
start_time = time.time()

for example in validation_set:
    predict(example)

end_time = time.time()

avg_time = (end_time - start_time) / n_samples
avg_time


## Conclusion

Le modèle fine-tuné a été évalué sur le jeu de validation SQuAD.

En plus des métriques Exact Match et F1, nous avons évalué les modèles
à l'aide des métriques Precision, Recall et AUC.

La courbe ROC permet d'analyser la capacité du modèle à distinguer
les réponses correctes des réponses incorrectes en fonction d'un
seuil de confiance.


In [ ]:
results_summary = {
    "model": "RoBERTa-base",
    "EM": results["exact_match"],
    "F1": results["f1"],
    "Precision": precision.mean(),
    "Recall": recall.mean(),
    "AUC": roc_auc,
    "Inference_time_ms": avg_time * 1000
}

results_summary


In [ ]:
import json

# Sauvegarder les résultats en JSON
with open("outputs/results_roberta.json", "w") as f:
    json.dump(results_summary, f, indent=2)

print("Résultats sauvegardés dans outputs/results_roberta.json")


## Résumé des résultats
